In [1]:
import main
from importlib import reload
reload(main)

<module 'main' from '/home/cabox/workspace/gleipnir/main.py'>

In [2]:
main.run("Proof_of_ownership.docx", file_out="Proof_decrypted_v03.docx")

KeyError: 'data'

In [1]:
import cProfile
from controller.blockchain import Blockchain
from controller import magic_key
from pymongo import MongoClient
from atlas.config import glpr_config as glc

server = MongoClient(glc.MONGODB)
db = server.gleipnir

In [13]:
pr = cProfile.Profile()
pr.enable()

file_in = "Proof_of_ownership.docx"
file_out = "Proof_decrypted_v03.docx"


# magic_key.generate_magic_keys("proof_mk.bin", "priv_key_proof.pem",
#                               "pub_key_proof.pem")

special_magic_key = magic_key.get_magic_key("proof_mk.bin",
                                            "priv_key_proof.pem")

blockchain = Blockchain(special_magic_key, db,
                        file_path=file_in, chunks=4)
splitted_data = blockchain._chunk_data()
blockchain.set_db(db)

pr.disable()
pr.print_stats(sort='time')

         123252 function calls (123218 primitive calls) in 0.776 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        4    0.174    0.044    0.174    0.044 {method 'recv' of '_socket.socket' objects}
        2    0.093    0.047    0.539    0.269 Primality.py:116(lucas_test)
    44342    0.077    0.000    0.077    0.000 {built-in method isinstance}
     2323    0.044    0.000    0.142    0.000 _Numbers_gmp.py:187(__init__)
     6194    0.040    0.000    0.048    0.000 _Numbers_gmp.py:492(__imod__)
     6138    0.036    0.000    0.051    0.000 _Numbers_gmp.py:473(__imul__)
     8184    0.035    0.000    0.051    0.000 _Numbers_gmp.py:639(set)
     4152    0.025    0.000    0.032    0.000 _Numbers_gmp.py:539(__irshift__)
     3097    0.023    0.000    0.034    0.000 _Numbers_gmp.py:617(multiply_accumulate)
     2418    0.019    0.000    0.035    0.000 api.py:225(new)
     2046    0.018    0.000    0.191    0.000 _Numbers_gm

        2    0.000    0.000    0.000    0.000 Primality.py:264(<listcomp>)
        2    0.000    0.000    0.000    0.000 threading.py:249(_is_owned)
        2    0.000    0.000    0.000    0.000 read_preferences.py:167(__ne__)
        2    0.000    0.000    0.000    0.000 _Numbers_gmp.py:600(is_perfect_square)
        1    0.000    0.000    0.000    0.000 <ipython-input-13-c2f547036a86>:19(<module>)
        2    0.000    0.000    0.000    0.000 son.py:217(__len__)
        1    0.000    0.000    0.000    0.000 _bootlocale.py:23(getpreferredencoding)
        1    0.000    0.000    0.000    0.000 {built-in method abs}
        3    0.000    0.000    0.000    0.000 {method 'startswith' of 'str' objects}
        2    0.000    0.000    0.000    0.000 message.py:178(<listcomp>)
        3    0.000    0.000    0.000    0.000 posixpath.py:38(_get_sep)
        2    0.000    0.000    0.000    0.000 topology.py:270(get_server_by_address)
        1    0.000    0.000    0.000    0.000 asn1.py:385(__in

In [14]:
pr = cProfile.Profile()


for bin_data in splitted_data:
    data = blockchain.encrypt_block(bin_data)
    previous_block = blockchain.get_previous_block()
    previous_txid = previous_block['transaction_id']
    pr.enable()
    new_block = blockchain.create_block(data, previous_txid, title=blockchain.file)
    pr.disable()
    pr.print_stats(sort='time')
#     if blockchain.check_prev_block_hash(new_block, previous_block):
#         db.BCMain.insert_one(new_block)

# collection = db.BCMain
# blockchain.convert_blocks_to_file(collection, file_name=file_out)

pr.disable()
pr.print_stats(sort='time')


         1564276 function calls in 223.227 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   142206  123.300    0.001  123.300    0.001 {built-in method openssl_sha256}
   142206   88.667    0.001   88.667    0.001 encoder.py:197(iterencode)
        1    2.653    2.653  223.227  223.227 blockchain.py:25(create_block)
   142206    2.002    0.000   91.608    0.001 encoder.py:175(encode)
   142206    1.820    0.000   93.898    0.001 __init__.py:182(dumps)
   142206    1.703    0.000    1.703    0.000 {method 'encode' of 'str' objects}
   142206    0.869    0.000   94.768    0.001 blockchain.py:87(format_dict_to_block)
   142206    0.804    0.000    0.804    0.000 {method 'hexdigest' of '_hashlib.HASH' objects}
   284413    0.711    0.000    0.711    0.000 {built-in method isinstance}
   142206    0.470    0.000    0.470    0.000 encoder.py:98(__init__)
   142206    0.229    0.000    0.229    0.000 {method 'join' of 'str' obje